In [15]:
import sys
print(sys.prefix)

/Users/eunbeejang/anaconda3


In [17]:
import torch
from torchtext import data
from torchtext import datasets
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemma = WordNetLemmatizer()

# Set the random seed manually for reproducibility.
torch.manual_seed(args.seed)
if torch.cuda.is_available():
    if not args.cuda:
        print("WARNING: You have a CUDA device, so you should probably run with --cuda")

ModuleNotFoundError: No module named 'torchtext'

In [11]:
def load_data():
    tokenize = lambda x: lemma.lemmatize(x.split())
    TEXT = data.Field(sequential=True, tokenize=tokenize, lower=True,
                      fix_length=200, include_lengths=True, batch_first=True)
    LABEL = data.LabelField(tensor_type=torch.FloatTensor)